# `Nuqleon.Reflection.Virtualization`

Provides interfaces for the .NET reflection APIs, enabling the construction of wrappers, alternative implementations, mocks, etc. One example of a wrapper over reflection is a memoizing cache to speed up expensive reflection operations.

## Reference the library

### Option 1 - Use a local build

If you have built the library locally, run the following cell to load the latest build.

In [ ]:
#r "bin/Debug/net6.0/Nuqleon.Reflection.Virtualization.dll"

### Option 2 - Use NuGet packages

If you want to use the latest published package from NuGet, run the following cell.

In [ ]:
#r "nuget:Nuqleon.Reflection.Virtualization,*-*"

## (Optional) Attach a debugger

If you'd like to step through the source code of the library while running samples, run the following cell, and follow instructions to start a debugger (e.g. Visual Studio). Navigate to the source code of the library to set breakpoints.

In [ ]:
System.Diagnostics.Debugger.Launch();

## `IReflectionProvider`

This library provides abstractions over the .NET reflection APIs using a whole bunch of fine-grained interfaces that represent different aspects of reflection. In the cell below, we reflect on the library to show all of the interfaces.

In [ ]:
using System.Reflection;

foreach (var t in typeof(IReflectionProvider).Assembly.GetTypes().Where(t => t.IsInterface).OrderBy(t => t.Name))
{
    var declaration = "interface " + t.Name;

    var ifs = t.GetInterfaces();
    if (ifs.Length > 0)
    {
        declaration += " : " + string.Join(", ", ifs.Select(t => t.Name));
    }

    declaration += ";";

    Console.WriteLine(declaration);
}

interface IAssemblyIntrospectionProvider;


interface IAssemblyLoadingProvider;


interface IComTypeLoadingProvider;


interface IEventInfoIntrospectionProvider : IMemberInfoIntrospectionProvider;


interface IFieldInfoIntrospectionProvider : IMemberInfoIntrospectionProvider;


...

interface IReflectionProvider : IReflectionIntrospectionProvider, IAssemblyIntrospectionProvider, IModuleIntrospectionProvider, ITypeIntrospectionProvider, IMemberInfoIntrospectionProvider, IReflectionTypeSystemProvider, IFieldInfoIntrospectionProvider, IMethodInfoIntrospectionProvider, IMethodBaseIntrospectionProvider, IPropertyInfoIntrospectionProvider, IEventInfoIntrospectionProvider, IParameterInfoIntrospectionProvider, IReflectionCreationProvider, ITypeCreationProvider, IMethodCreationProvider, IReflectionInvocationProvider, IReflectionLoadingProvider, IAssemblyLoadingProvider, IModuleLoadingProvider, ITypeLoadingProvider, IComTypeLoadingProvider, IReflectionHandlerResolver;


interface IReflectionTypeSystemProvider;


interface ITypeCreationProvider;


interface ITypeIntrospectionProvider : IMemberInfoIntrospectionProvider, IReflectionTypeSystemProvider;


interface ITypeLoadingProvider : IComTypeLoadingProvider;


The presence of fine-grained interfaces enables components to take a dependency on just the aspects of reflection they rely on. At the top of the interface hierarchy, the `IReflectionProvider` interface unions all of these aspects together.

Two implementations of `IReflectionProvider` are provided in the library:

* `DefaultReflectionProvider` implements the interface using the built-in .NET reflection APIs, providing virtual methods that can be overridden.
* `CachingDefaultReflectionProvider` which uses memoization techniques from `Nuqleon.Memory` to speed up repeated reflection API invocations.

Using these base classes, one can alter reflection behavior by overriding methods, e.g. to intercept calls to `GetType` used to load a type by name. This is used in `Reaqtor` to redirect types that have been relocated to other assemblies or even have been renamed.

In the cell below, we'll just have a quick look at the `DefaultReflectionProvider` to call some reflection API.

In [ ]:
IReflectionProvider defaultProvider = DefaultReflectionProvider.Instance;

IReadOnlyList<Type> listOfInt = defaultProvider.GetGenericArguments(typeof(Dictionary<string, int>));

Console.WriteLine(string.Join(", ", listOfInt));

System.String, System.Int32


Note that the return type of `GetGenericArguments` on the interface is not `Type[]` but `IReadOnlyList<Type>`. This differs from the reflection APIs but matches the 99.9% use case of these APIs, where the caller does not mutate the resulting array of types. This in turn enables us to build caching reflection providers that can avoid the allocation cost. In the cell below, we show the use of the `CachingReflectionProvider`.

In [ ]:
using System.Memory;

IMemoizer mem = Memoizer.Create(MemoizationCacheFactory.Unbounded);

IReflectionProvider cachingProvider = new CachingDefaultReflectionProvider(mem);

IReadOnlyList<Type> listOfInt = cachingProvider.GetGenericArguments(typeof(Dictionary<string, int>));

Console.WriteLine(string.Join(", ", listOfInt));

System.String, System.Int32


To show the effect of caching, let's refactor the code a tiny bit.

In [ ]:
static void Test(IReflectionProvider provider) => provider.GetGenericArguments(typeof(Dictionary<string, int>));

We'll also leverage `Nuqleon.Time` facilities to measure elapsed time and memory allocations.

In [ ]:
using System.Time;

class MemoryClock : IClock
{
    public long Now => GC.GetAllocatedBytesForCurrentThread();
}

IStopwatch swMem = StopwatchFactory.FromClock(new MemoryClock()).Create();
IStopwatch swTime = StopwatchFactory.Diagnostics.Create();

With the test code and stopwatches in place, we can write a quick benchmark utility.

In [ ]:
void Benchmark(IReflectionProvider provider, int n)
{
    swMem.Restart();
    swTime.Restart();
    {
        for (int i = 0; i < n; i++)
        {
            Test(provider);
        }
    }
    swTime.Stop();
    swMem.Stop();
}

foreach (var (name, provider) in new[]
{
    ("default", defaultProvider),
    ("caching", cachingProvider),
})
{
    Console.WriteLine(name);
    Benchmark(provider, 1_000_000);
    Console.WriteLine($"{swMem.ElapsedTicks} bytes allocated; {swTime.ElapsedMilliseconds} ms spent.");
    Console.WriteLine();
}

default


40000120 bytes allocated; 107 ms spent.


caching


288 bytes allocated; 75 ms spent.
